In [ ]:
import sqlite3
import pandas as pd

In [ ]:
DB_PATH = "/resources/db/data.db"

In [ ]:
def query_data(sql: str, database: str = DB_PATH) -> list[dict]:
    """
    Executes a SQL query on the specified SQLite database and returns the results as a list of dictionaries.

    Args:
        sql (str): The SQL query to execute.
        database (str): The path to the SQLite database file.

    Returns:
        list[dict]: A list of dictionaries representing the query results, where each dictionary corresponds to a row.
                    If an error occurs, returns a string with the error message.
    """
    conn = sqlite3.connect(database)

    try:
        result = conn.execute(sql).fetchall()
        conn.commit()
        # Get column names from cursor description
        columns = [desc[0] for desc in conn.execute(sql).description]
        # Convert rows to list of dicts
        return [dict(zip(columns, row)) for row in result]

    except Exception as e:
        return f"Error: {str(e)}"
    finally:
        conn.close()

In [ ]:
query = "SELECT * FROM bloques_diputados"

In [ ]:
query_data(query)

In [ ]:
query = """
    SELECT
        bloqueId, voto, COUNT(*) AS cantidad
    FROM votaciones_diputados
    WHERE bloqueId = (SELECT bloqueId FROM bloques_diputados WHERE bloque = 'Unión por la Patria')
        AND asuntoId = (SELECT asuntoID FROM asuntos_diputados WHERE ASUNTO = 'O.D. 7 - Ley de Bases. Votación en particular. TITULO X. ARTS. 228 AL 231.')
    GROUP BY bloqueId, voto
"""

In [ ]:
query_data(query)

In [ ]:
query = """
    SELECT
        diputadoId, voto, count(*) AS cantidad
    FROM votaciones_diputados
    WHERE diputadoId = (SELECT diputadoId FROM legisladores_diputados WHERE nombre = 'MILEI, Javier Gerardo')
    GROUP BY diputadoId, voto
"""

In [ ]:
query_data(query)

---

In [ ]:
from pydantic_ai import Agent, RunContext
from pydantic_ai.settings import ModelSettings

from cparla.retriever import Retriever

In [ ]:
# Example: conf/llm_agents/deputy_votes_agent.yaml
model = "openai:gpt-4.1-2025-04-14"
max_tokens = 1024
temperature = 0.0

instructions_template = """
    You are an expert assistant for answering questions about votes in the Argentine Chamber of Deputies.
    You must use the provided tools to answer user queries. Do NOT use your own knowledge or speculate.

    ## Tools:
    - `retrieve_deputy`: Given a query or a deputy's name, returns possible matches from the "legisladores_diputados" collection, including their diputadoId and metadata.
    - `query_db`: Executes a SQL query on the database and returns the results.

    ## Guidelines:
    - For questions about a deputy's votes, first use `retrieve_deputy` to get the correct diputadoId(s).
    - Then, use `query_db` to fetch the votes from the "votaciones_diputados" table using the diputadoId(s).
    - If the query is ambiguous, return all possible matches from `retrieve_deputy`.
    - Do NOT answer unless you have used the tools to get the information.
    - Respond in your own words, do not refer to the tools or their names in the answer.

    ## Example:
    - User: "¿Cuáles fueron los votos de Myriam Bregman?"
    - Steps:
        1. Use `retrieve_deputy` with "Myriam Bregman" → get diputadoId(s).
        2. Use `query_db` with "SELECT * FROM votaciones_diputados WHERE diputadoId IN (...)".
        3. Summarize the results for the user.

    ## Output:
    - Provide a clear answer to the user's question, based only on the tool results.
    - If no results are found, say so.

    ALWAYS respond in Spanish.
"""

In [ ]:
diputados_agent = Agent(
    model=model,
    # output_type=output_type,
    # deps_type=deps_type,
    name="Diputados Votes Agent",
    model_settings=ModelSettings(max_tokens=max_tokens, temperature=temperature),
    retries=1,
    # tools=tools,
    # mcp_servers=mcp_servers,  # type: ignore
    # history_processors=history_processors,
)

In [ ]:
from functools import lru_cache
from langchain_openai import OpenAIEmbeddings


@lru_cache()
def get_openai_embeddings(
    model: str = "text-embedding-3-large",
    dimensions: int = 256,
) -> OpenAIEmbeddings:
    return OpenAIEmbeddings(
        model=model,
        dimensions=dimensions,
    )


dense_embeddings = get_openai_embeddings()
dense_embeddings

In [ ]:
retriever = Retriever(dense_embeddings=dense_embeddings)

In [ ]:
@diputados_agent.tool
async def retrieve_deputy(
    ctx: RunContext,
    query: str,
    collection_name: str = "legisladores-diputados",
) -> list[dict]:
    """
    Uses the embedding retriever to find deputies matching the query.
    Returns a list of dicts with at least 'diputadoId', 'nombre', and other metadata.
    """
    # Example using your Retriever class
    results = await retriever.hybrid_search(
        collection_name=collection_name,
        query=query,
        k=10,
    )

    return results

In [ ]:
@diputados_agent.tool
def query_db(
    ctx: RunContext,
    sql: str,
) -> list[dict]:
    """
    Executes a SQL query on the database and returns the results as a list of dictionaries.
    """
    return query_data(sql)

In [ ]:
await diputados_agent.run("¿Cuáles fueron los votos de Javier Milei?")

In [ ]:
import logfire

logfire.configure(service_name="modulo-consultas-parlamentarias")
_ = logfire.instrument_pydantic_ai()
_ = logfire.instrument_openai()

In [ ]:
await diputados_agent.run("¿Cuáles fueron los votos de Javier Milei?")

---

In [ ]:
# Example: conf/llm_agents/deputy_votes_agent.yaml
model = "openai:gpt-4.1-2025-04-14"
max_tokens = 1024
temperature = 0.0

instructions_template = """
    You are an expert assistant for answering questions about votes in the Argentine Chamber of Deputies.
    You must use the provided tools to answer user queries. Do NOT use your own knowledge or speculate.

    ## Database Schema:
    The following tables are available in the database:

    - **legisladores_diputados**
        - diputadoId (INTEGER)
        - nombre (TEXT)
        - distrito (TEXT)
        - sexo (TEXT)
        - imagen (TEXT)

    - **votaciones_diputados**
        - asuntoId (INTEGER)
        - diputadoId (INTEGER)
        - bloqueId (INTEGER)
        - voto (TEXT)

    - **asuntos_diputados**
        - asuntoId (INTEGER)
        - sesion (TEXT)
        - asunto (TEXT)
        - ano (INTEGER)
        - fecha (TEXT)
        - hora (TEXT)
        - base (TEXT)
        - mayoria (TEXT)
        - resultado (TEXT)
        - presidente (TEXT)
        - presentes (INTEGER)
        - ausentes (INTEGER)
        - abstenciones (INTEGER)
        - afirmativos (INTEGER)
        - negativos (INTEGER)
        - votopresidente (TEXT)
        - titulo (TEXT)
        - auditoria (TEXT)

    - **bloques_diputados**
        - bloqueId (INTEGER)
        - bloque (TEXT)
        - color (TEXT)

    ## Tools:
    - `retrieve_deputy`: Given a query or a deputy's name, returns possible matches from the "legisladores_diputados" collection, including their diputadoId and metadata.
    - `query_db`: Executes a SQL query on the database and returns the results.

    ## Guidelines:
    - For questions about a deputy's votes, first use `retrieve_deputy` to get the correct diputadoId(s).
    - Then, use `query_db` to fetch the votes from the "votaciones_diputados" table using the diputadoId(s).
    - Use only the tables and columns listed above. Do NOT invent table or column names.
    - If the query is ambiguous, return all possible matches from `retrieve_deputy`.
    - Do NOT answer unless you have used the tools to get the information.
    - Respond in your own words, do not refer to the tools or their names in the answer.

    ## Example:
    - User: "¿Cuáles fueron los votos de Myriam Bregman?"
    - Steps:
        1. Use `retrieve_deputy` with "Myriam Bregman" → get diputadoId(s).
        2. Use `query_db` with "SELECT * FROM votaciones_diputados WHERE diputadoId IN (...)".
        3. Summarize the results for the user.

    ALWAYS respond in Spanish.
"""

In [ ]:
diputados_agent = Agent(
    model=model,
    # output_type=output_type,
    # deps_type=deps_type,
    instructions=instructions_template,
    name="Diputados Votes Agent",
    model_settings=ModelSettings(max_tokens=max_tokens, temperature=temperature),
    retries=1,
    # tools=tools,
    # mcp_servers=mcp_servers,  # type: ignore
    # history_processors=history_processors,
)

In [ ]:
from functools import lru_cache
from langchain_openai import OpenAIEmbeddings


@lru_cache()
def get_openai_embeddings(
    model: str = "text-embedding-3-large",
    dimensions: int = 256,
) -> OpenAIEmbeddings:
    return OpenAIEmbeddings(
        model=model,
        dimensions=dimensions,
    )


dense_embeddings = get_openai_embeddings()
dense_embeddings

In [ ]:
retriever = Retriever(dense_embeddings=dense_embeddings)

In [ ]:
@diputados_agent.tool
async def retrieve_deputy(
    ctx: RunContext,
    query: str,
    collection_name: str = "legisladores-diputados",
) -> list[dict]:
    """
    Uses the embedding retriever to find deputies matching the query.
    Returns a list of dicts with at least 'diputadoId', 'nombre', and other metadata.
    """
    # Example using your Retriever class
    results = await retriever.hybrid_search(
        collection_name=collection_name,
        query=query,
        k=10,
    )

    return results

In [ ]:
@diputados_agent.tool
def query_db(
    ctx: RunContext,
    sql: str,
) -> list[dict]:
    """
    Executes a SQL query on the database and returns the results as a list of dictionaries.
    """
    return query_data(sql)

In [ ]:
await diputados_agent.run("¿Cuáles fueron los votos de Javier Milei?")

In [ ]:
query = """
    SELECT
        vd.voto, ad.titulo, ad.fecha
    FROM votaciones_diputados vd
    JOIN asuntos_diputados ad ON vd.asuntoId = ad.asuntoId
    WHERE vd.diputadoId = 1877 ORDER BY ad.fecha DESC
"""

In [ ]:
results = pd.DataFrame(query_data(query))
results

In [ ]:
result = await diputados_agent.run("¿Cuáles fueron los votos de Javier Milei?")
result

In [ ]:
# query = "¿Cuáles fueron los votos de Javier Milei?"
print(result.output)

In [ ]:
results.iloc[85]["titulo"]

In [ ]:
results["fecha"] = pd.to_datetime(results["fecha"], format="mixed")
results.loc[results["voto"] == "AFIRMATIVO"].sort_values(by="fecha", ascending=False)

In [ ]:
result = await diputados_agent.run(
    "¿Podrás indicarme el resumen de los votos de Javier Milei, indicando la cantidad de votos afirmativos, negativos, abstenciones y ausencias?"
)

In [ ]:
# query = "¿Podrás indicarme el resumen de los votos de Javier Milei, indicando la cantidad de votos afirmativos, negativos, abstenciones y ausencias?"
print(result.output)

In [ ]:
# Chequeo
query = """
    SELECT
        vd.voto, COUNT(*) AS cantidad
    FROM votaciones_diputados vd
    WHERE vd.diputadoId = 1877
    GROUP BY vd.voto
"""

pd.DataFrame(query_data(query))

In [ ]:
# query = "¿Cuáles fueron los votos de Javier Milei?"
print(result.output)

In [ ]:
pd.set_option("display.max_colwidth", None)

In [ ]:
# Chequeo
query = """
    SELECT
        vd.voto, ad.fecha, ad.titulo, ad.asunto
    FROM votaciones_diputados vd
    LEFT JOIN asuntos_diputados ad ON vd.asuntoId = ad.asuntoId
    WHERE vd.diputadoId = 1877 AND vd.voto = 'NEGATIVO'
"""

result = pd.DataFrame(query_data(query))
result["fecha"] = pd.to_datetime(result["fecha"], format="mixed")
result.sort_values(by=["fecha", "asunto"], ascending=[False, True])[["fecha", "titulo"]]

In [ ]:
result.groupby("fecha").size().sort_index()

In [ ]:
votos_milei = pd.read_csv(
    "/resources/data/Estadística - MILEI, JAVIER GERARDO - Total.csv"
)
votos_milei

In [ ]:
votos_milei["fecha"] = pd.to_datetime(votos_milei["FECHA"], format="mixed").dt.date
votos_milei.loc[votos_milei["¿CÓMO VOTÓ?"] == "NEGATIVO"].groupby(
    "fecha"
).size().sort_index()

In [ ]:
result = await diputados_agent.run(
    "¿Podrás indicarme el resumen de los votos de José Luis Espert, desglosando la cantidad de votos afirmativos, negativos, abstenciones y ausencias?"
)

In [ ]:
print(result.output)

In [ ]:
result = await diputados_agent.run(
    "¿Podrás indicarme el resumen de los votos de vICTORIA ViLLARUEKL, desglosando la cantidad de votos afirmativos, negativos, abstenciones y ausencias?"
)

In [ ]:
print(result.output)